# OPS-SAT case starter-kit notebook

ESA's [Kelvins](https://kelvins.esa.int) competition "[the OPS-SAT case](https://kelvins.esa.int/opssat/home/)" is a novel data-centric challenge that asks you to work with the raw data of a satellite and very few provided labels to find the best parameters for a given machine learning model. Compared to previous competitions on Kelvins (like the [Pose Estimation](https://kelvins.esa.int/pose-estimation-2021/) or the [Proba-V Super-resolution challenge](https://kelvins.esa.int/proba-v-super-resolution/)) where the test-set is provided and the infered results are submitted, for the OPS-SAT case, we will run inference on the Kelvins server directly! This notebooks contains examples on how you can load your data and train an **EfficientNetLite0** model by only using the 80-labeled images provided. Therefore, the directory `images`, containing unlabeld patches and included in the training dataset is not used for this notebook. However, competitors are encouraged to use these patches to improve the model accuracy.

# 1. Module imports

If you do not have a GPU, uncomment and run the next commands.


In [1]:
import sys
sys.version

'3.9.16 (main, Mar  8 2023, 14:00:05) \n[GCC 11.2.0]'

In [2]:
! python --version

/usr/bin/zsh: /home/ramez/miniconda3/envs/thesis/lib/libtinfo.so.6: no version information available (required by /usr/bin/zsh)
Python 3.9.16


In [3]:
! which python

/usr/bin/zsh: /home/ramez/miniconda3/envs/thesis/lib/libtinfo.so.6: no version information available (required by /usr/bin/zsh)
/home/ramez/miniconda3/envs/thesis/bin/python


In [4]:
import tensorflow as tf

#### GPU Check

In [5]:
!python3 -c "import tensorflow as tf; print(tf.config.list_physical_devices('GPU'))"

/usr/bin/zsh: /home/ramez/miniconda3/envs/thesis/lib/libtinfo.so.6: no version information available (required by /usr/bin/zsh)
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [6]:
from tensorflow import keras

In [7]:
import numpy as np

In [8]:
from sklearn.metrics import cohen_kappa_score

In [9]:
## Local EfficientNetLite (Customized by the Competition)
from efficientnet_lite import EfficientNetLiteB0

In [10]:
from sklearn.model_selection import KFold

In [11]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout

In [12]:
from tensorflow.keras.callbacks import TensorBoard

In [13]:
from keras.callbacks import EarlyStopping, ModelCheckpoint

In [14]:
import matplotlib.pyplot as plt

In [15]:
from itertools import accumulate

In [16]:
import os

In [17]:
import wandb
from wandb.keras import WandbMetricsLogger, WandbModelCheckpoint

# 2. Utility Functions

You can use this function to load your training data.

In [18]:
def get_images_from_path(dataset_path):
    """ Get images from path and normalize them applying channel-level normalization. """

    # loading all images in one large batch
    tf_eval_data = tf.keras.utils.image_dataset_from_directory(dataset_path, image_size=config.input_shape[:2], shuffle=False, 
                                                               batch_size=100000, label_mode='int')

    # get the class names (folder names) from the dataset object
    global class_names
    class_names = tf_eval_data.class_names
    
    # extract images and targets
    for tf_eval_images, tf_eval_targets in tf_eval_data:
        break

    return tf.convert_to_tensor(tf_eval_images), tf_eval_targets

# 4. Loading data

In [19]:
# dataset_path_train="/home/ramez/Politechnika_Slaska_MSc/Thesis/Competition/Data/images_copy_processed/"
dataset_path_train_val = "/home/ramez/Politechnika_Slaska_MSc/Thesis/Competition/Data/ops_sat_train/"

In [20]:
dataset_path_test = "/home/ramez/Politechnika_Slaska_MSc/Thesis/Competition/Data/ops_sat_test/"

In this notebook, classical supervised learning is used. Therefore, remember to remove the subdirectory `images` containing unlabeled patches before loading the dataset to perform training correctly.

### Model Configuration

In [21]:
class CustomLoss(tf.keras.losses.Loss):
    def __init__(self, **kwargs):
        super(CustomLoss, self).__init__(**kwargs)
        self.scce = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)

    def call(self, y_true, y_pred):
        return self.scce(y_true, y_pred)

In [22]:
custom_loss_functions = {
    'SparseCategoricalCrossentropy': CustomLoss
}

In [23]:
# Start a run, tracking hyperparameters
wandb.init(
    # set the wandb project where this run will be logged
    project="OPS-SAT-Thesis-Project",

    # track hyperparameters and run metadata with wandb.config
    config={
        "dropout": 0.5,
        "num_classes": 8,
        "input_shape": (200, 200, 3),
        "model_weights": "imagenet",
        "output_layer_activation": "softmax",
        "model_optimizer": "adam",
        # "loss_fun": "sparse_categorical_crossentropy",
        "loss_fun": "SparseCategoricalCrossentropy",
        "model_metrics": ["SparseCategoricalAccuracy"],
        "early_stopping_monitor": "val_sparse_categorical_accuracy",
        "early_stopping_patience": 3,
        "model_checkpoint_monitor": "val_sparse_categorical_accuracy",
        "cross_validation_k": 5,
        "model_epochs": 100,
        "model_batch_size": 8
    }
)

config = wandb.config

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: ramezashendy (ramez). Use `wandb login --relogin` to force relogin


In [24]:
#Loading dataset
x_train_val, y_train_val = get_images_from_path(dataset_path_train_val)

Found 227 files belonging to 8 classes.


In [25]:
## class_names_labels dict
class_name_label = {}
for i, class_name in enumerate(class_names):
    class_name_label[i] = class_name

In [26]:
#Loading dataset
x_test, y_test = get_images_from_path(dataset_path_test)

Found 40 files belonging to 8 classes.


In [27]:
## class_names_labels dict
class_name_label_test = {}
for i, class_name in enumerate(class_names):
    class_name_label_test[i] = class_name

In [28]:
## Check if classes are identical with labels in both splits
if class_name_label == class_name_label_test:
    print("Ok")
else:
    print("Error - Class Mapping Are Mismatched")

Ok


# 5. Model training

The network architecture used for OPS-SAT is **EfficientNetLite0**. We would like to thank Sebastian for making a Keras implementation of EfficientNetLite publicly available under the Apache 2.0 License: https://github.com/sebastian-sz/efficientnet-lite-keras. Our Version of this code has been modified to better fit our purposes. For example, we removed the ReLU "stem_activation" to better match a related efficientnet pytorch implementation. In any way, **you have to use the model architecture that we provide in our [starter-kit](https://gitlab.com/EuropeanSpaceAgency/the_opssat_case_starter_kit).**

### Load The Model With ImageNet Weights

### Compile The model

### Early Stopping and Best Model Callbacks 

We provide now an example on how you can train your model by using standard supervised learning. Training loss (`SparseCategoricalCrossentropy`) and `Accuracy` are shown for simplicity and for an easier interpretation of the training outcome, despite your submission will be evaluated by using the metric **1 - Cohen's kappa** [metric](https://en.wikipedia.org/wiki/Cohen's_kappa). For more information on scoring, please refer to [Scoring](https://kelvins.esa.int/opssat/scoring/).

In [29]:
custom_loss = custom_loss_functions[config.loss_fun]()

In [30]:
def model_init():
    global model
    global early_stopping
    global checkpoint
    
    model = EfficientNetLiteB0(classes=config.num_classes, weights=config.model_weights, input_shape=config.input_shape, classifier_activation=None, include_top = False)
    x = model.output
    x = GlobalAveragePooling2D()(x)
    x = Dropout(config.dropout)(x)
    output_layer = Dense(config.num_classes, activation=config.output_layer_activation)(x)
    model = Model(inputs=model.input, outputs=output_layer)
    # model.summary()
    
    model.compile(optimizer=config.model_optimizer,
                  # loss="sparse_categorical_crossentropy",
              # loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False), 
                  # loss=config.loss_fun,
                  loss=custom_loss,
                  # metrics=[keras.metrics.SparseCategoricalAccuracy()],
                 metrics=config.model_metrics)
    
    early_stopping = EarlyStopping(monitor=config.early_stopping_monitor, patience=config.early_stopping_patience)
    checkpoint = ModelCheckpoint('best_weights.h5', monitor=config.model_checkpoint_monitor, save_best_only=True)
    
# wandb_callback=wandb.keras.WandbCallback()

### Without K fold Cross Validation but with TensorBoard (With Early Stopping)

In [31]:
# tensorboard_callback = TensorBoard(log_dir='./logs')

In [32]:
# early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
# history = model.fit(x_train_val, y_train_val, validation_data=(x_test, y_test), epochs= 100, verbose=1, batch_size=8, 
#                         callbacks=[early_stopping, tensorboard_callback])

In [33]:
# !tensorboard --logdir=./logs

### With K fold Cross Validation but without TensorBoard (With Early Stopping)

In [34]:
kf = KFold(n_splits=config.cross_validation_k, shuffle=True)

In [35]:
# Train and evaluate the model using K-fold cross-validation
scores = []
training_accuracy = []
validation_accuracy = []
training_loss = []
validation_loss = []

## Initializing the model
model_init()

## counter for folds
i = 1

for train_idx, val_idx in kf.split(x_train_val):
    X_train = tf.gather(x_train_val, train_idx)
    y_train = tf.gather(y_train_val, train_idx)
    X_val = tf.gather(x_train_val, val_idx)
    y_val = tf.gather(y_train_val, val_idx)
    
    history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs= config.model_epochs, verbose=1, batch_size=config.model_batch_size, 
                        callbacks=[early_stopping, checkpoint])

    training_accuracy.append(history.history['sparse_categorical_accuracy'])
    validation_accuracy.append(history.history['val_sparse_categorical_accuracy'])
    training_loss.append(history.history['loss'])
    validation_loss.append(history.history['val_loss'])
    
    model.load_weights('best_weights.h5')
    score = model.evaluate(X_val, y_val)
    scores.append(score[1])
    
    wandb.log({"Best Validation Loss for Folds": score[0]})
    wandb.log({"Best Validation Accuracy for Folds" : score[1]})
    
    # Define the current and new file names
    current_name = 'best_weights.h5'
    new_name = 'fold_' + str(i) + '_best_model_weights.h5'
    
    i+=1

    # Rename the file
    os.rename(current_name, new_name)
    
    ## Reseting the model for the next fold
    model_init()

block1a_ same
block2a_ ((1, 1), (1, 1))
block2a_ valid
block2b_ same
block3a_ ((2, 2), (2, 2))
block3a_ valid
block3b_ same
block4a_ ((1, 1), (1, 1))
block4a_ valid
block4b_ same
block4c_ same
block5a_ same
block5b_ same
block5c_ same
block6a_ ((2, 2), (2, 2))
block6a_ valid
block6b_ same
block6c_ same
block6d_ same
block7a_ same
Epoch 1/100


2023-04-27 19:59:00.669815: W tensorflow/stream_executor/gpu/asm_compiler.cc:111] *** WARNING *** You are using ptxas 9.2.148, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

You may not need to update to CUDA 11.1; cherry-picking the ptxas binary is often sufficient.


23/23 [==============================] - ETA: 0s - loss: 1.6192 - sparse_categorical_accuracy: 0.4309

/home/ramez/miniconda3/envs/thesis/lib/python3.9/site-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)


23/23 [==============================] - 10s 214ms/step - loss: 1.6192 - sparse_categorical_accuracy: 0.4309 - val_loss: 3.1707 - val_sparse_categorical_accuracy: 0.3043
Epoch 2/100
23/23 [==============================] - 3s 119ms/step - loss: 0.7720 - sparse_categorical_accuracy: 0.7790 - val_loss: 4.6626 - val_sparse_categorical_accuracy: 0.3043
Epoch 3/100
23/23 [==============================] - 3s 143ms/step - loss: 0.6987 - sparse_categorical_accuracy: 0.7514 - val_loss: 3.6246 - val_sparse_categorical_accuracy: 0.4348
Epoch 4/100
23/23 [==============================] - 3s 143ms/step - loss: 0.5401 - sparse_categorical_accuracy: 0.8066 - val_loss: 2.2868 - val_sparse_categorical_accuracy: 0.5870
Epoch 5/100
23/23 [==============================] - 3s 139ms/step - loss: 0.5423 - sparse_categorical_accuracy: 0.8343 - val_loss: 1.4785 - val_sparse_categorical_accuracy: 0.6304
Epoch 6/100
23/23 [==============================] - 3s 122ms/step - loss: 0.5009 - sparse_categorical_acc

In [ ]:
# Print the mean validation accuracy
print('Validation accuracy mean: {:.2f} (std {:.2f})'.format(np.mean(scores), np.std(scores)))

In [ ]:
scores

Calculating the **1 - Cohen's kappa** score of the trained model on the trained dataset.

In [ ]:
## select the model with the best fold's validation accuracy
maximum = max(scores)
index_of_maximum = scores.index(maximum)
best_fold = index_of_maximum + 1

model.load_weights('fold_' + str(best_fold) + '_best_model_weights.h5')

In [ ]:
predictions = np.zeros(len(y_test), dtype=np.int8)
# inference loop
for e, (image, target) in enumerate(zip(x_test, y_test)):
    image = np.expand_dims(np.array(image), axis=0)
    output = model.predict(image)
    predictions[e] = np.squeeze(output).argmax()
#Keras model score
score_keras = 1 - cohen_kappa_score(y_test.numpy(), predictions)
print("Score:",score_keras)

### Accuracy 

In [ ]:
# assuming you have two lists, predicted and actual
correct = 0
total = len(y_test.numpy())

for pred, label in zip(predictions, y_test.numpy()):
    if pred == label:
        correct += 1

accuracy = correct / total

print("Accuracy: ", accuracy)

## Train - Validation Visualization

In [ ]:
eps_per_fold = [len(sub_list) for sub_list in training_accuracy]
eps_per_fold

In [ ]:
eps_per_fold_cum = list(accumulate(eps_per_fold))

In [ ]:
training_accuracy = [element for sublist in training_accuracy for element in sublist]
validation_accuracy = [element for sublist in validation_accuracy for element in sublist]
training_loss = [element for sublist in training_loss for element in sublist]
validation_loss = [element for sublist in validation_loss for element in sublist]

In [ ]:
eps_per_fold_cum

In [ ]:
xs = []
ys = []
for i in range(config.cross_validation_k):
    xs.append(eps_per_fold_cum[i]+1)
    ys.append(training_accuracy[eps_per_fold_cum[0]-1])
# xs = [eps_per_fold_cum[0]+1, eps_per_fold_cum[1]+1, eps_per_fold_cum[2]+1, eps_per_fold_cum[3]+1, eps_per_fold_cum[4]+1]
# ys = [training_accuracy[eps_per_fold_cum[0]-1], training_accuracy[eps_per_fold_cum[1]-1], training_accuracy[eps_per_fold_cum[2]-1],
#          training_accuracy[eps_per_fold_cum[3]-1], training_accuracy[eps_per_fold_cum[4]-1]]

In [ ]:
x1 = list(range(1, len(training_accuracy) + 1))
x2 = list(range(1, len(validation_accuracy) + 1))

fig = plt.figure(figsize=(15, 6))

# Create the plot
plt.plot(x1, training_accuracy, label='Training Accuracy')
plt.plot(x2, validation_accuracy, label='Validation Accuracy')



# Add points with labels
for i in range(config.cross_validation_k-1):
    plt.axvline(x=xs[i], color='red', linestyle='--')
# plt.axvline(x=xs[0], color='red', linestyle='--')
# plt.axvline(x=xs[1], color='red', linestyle='--')
# plt.axvline(x=xs[2], color='red', linestyle='--')
# plt.axvline(x=xs[3], color='red', linestyle='--')

# Add annotations to the lines
for i in range(config.cross_validation_k-1):
    plt.annotate('Fold '+str(i+2), xy=(xs[i], 0.2), xytext=(xs[i]+1, 0.15),
             arrowprops=dict(facecolor='black', arrowstyle='->'))

# plt.annotate('Fold 2', xy=(xs[0], 0.5), xytext=(xs[0]+1, 0.4),
#              arrowprops=dict(facecolor='black', arrowstyle='->'))
# plt.annotate('Fold 3', xy=(xs[1], 0.5), xytext=(xs[1]+1, 0.4),
#              arrowprops=dict(facecolor='black', arrowstyle='->'))
# plt.annotate('Fold 4', xy=(xs[2], 0.5), xytext=(xs[2]+1, 0.4),
#              arrowprops=dict(facecolor='black', arrowstyle='->'))
# plt.annotate('Fold 5', xy=(xs[3], 0.5), xytext=(xs[3]+1, 0.4),
#              arrowprops=dict(facecolor='black', arrowstyle='->'))
# plt.axvline(x=xs[4], color='red', linestyle='--')
# plt.scatter(xs, ys, color='red')
# plt.annotate('Point 1', (2, 4), textcoords="offset points", xytext=(0,10), ha='center')
# plt.annotate('Point 2', (4, 8), textcoords="offset points", xytext=(0,10), ha='center')


# Add a legend
plt.legend()

# Enable the grid
# plt.grid(True)

# Show the plot
plt.show()

## Log Unified Test Set and its predictions

In [ ]:
prediction_names = [class_name_label[num] for num in predictions]
actual_names = [class_name_label[num] for num in y_test.numpy()]

In [ ]:
# assuming you have lists of images, predictions, and actual labels
for i, (img, pred, label) in enumerate(zip(x_test, prediction_names, actual_names)):
    wandb.log({'All Predictions - Unified Test Set': wandb.Image(img, caption='predicted: {}, actual: {}'.format(pred, label))})

In [ ]:
pred_is_actual = [x == y for x, y in zip(prediction_names, actual_names)]

In [ ]:
wandb.finish()

In [ ]:
## Load the Model without any weight initialization
# model = EfficientNetLiteB0(classes=num_classes, weights=None, input_shape=input_shape, classifier_activation=None)
# model.summary()

In [ ]:
## Saving model
# model.save_weights('/home/ramez/Politechnika_Slaska_MSc/Thesis/Competition/submission/submit_19.h5')

In [ ]:
# model = EfficientNetLiteB0(classes=num_classes, weights=None, input_shape=input_shape, classifier_activation=None)
# # model = EfficientNetLiteB0(include_top=False, classes=num_classes, weights='imagenet', input_shape=input_shape, classifier_activation=None)
# model.load_weights('/home/ramez/Politechnika_Slaska_MSc/Thesis/Competition/submission/model_patterns_20epochs.h5')

The model will be now compiled and tested again. You should get the same score as before saving and loading. 

In [ ]:
# #Model shall be compiled before testing.
# model.compile()

# #Creating empty predictions
# predictions = np.zeros(len(y_test), dtype=np.int8)
# # inference loop
# for e, (image, target) in enumerate(zip(x_test, y_test)):
#     image = np.expand_dims(np.array(image), axis=0)
#     output = model.predict(image)
#     predictions[e] = np.squeeze(output).argmax()
# #Keras model score
# score_keras = 1 - cohen_kappa_score(y_test.numpy(), predictions)
# print("Score:",score_keras)